<a href="https://colab.research.google.com/github/bsbatusesli/CFU-Playground/blob/main/amaranth/SIMD_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Amaranth 
!pip install --upgrade 'amaranth[builtin-yosys]'

# CFU-Playground library
!git clone https://github.com/google/CFU-Playground.git
import sys
sys.path.append('CFU-Playground/python')

# Imports
from amaranth import *
from amaranth.back import verilog
from amaranth.sim import Delay, Simulator, Tick
from amaranth_cfu import TestBase, SimpleElaboratable, pack_vals, simple_cfu, InstructionBase, CfuTestBase
import re, unittest

# Utility to convert Amaranth to verilog 
def convert_elaboratable(elaboratable):
  v = verilog.convert(elaboratable, name='Top', ports=elaboratable.ports)
  v = re.sub(r'\(\*.*\*\)', '', v)
  return re.sub(r'^ *\n', '\n', v, flags=re.MULTILINE)

def runTests(klazz):
  loader = unittest.TestLoader()
  suite = unittest.TestSuite()
  suite.addTests(loader.loadTestsFromTestCase(klazz))
  runner = unittest.TextTestRunner()
  runner.run(suite)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'CFU-Playground' already exists and is not an empty directory.


In [180]:
#function for codebook storing. 
#Codebook has 4 clusters => 2bit codewords


class SIMD8_StoreCodebook(SimpleElaboratable):
  def __init__(self):
    # 2 x 32 bit input lines
    self.in0 = Signal(32)
    self.in1 = Signal(32)
    self.enable = Signal()
    self.clear = Signal()
    self.clus0 = Signal(signed(8))
    self.clus1 = Signal(signed(8))
    self.clus2 = Signal(signed(8))
    self.clus3 = Signal(signed(8))

  def elab(self,m):
    with m.If(self.enable):
        m.d.comb +=[self.clus0.eq(self.in1[0:8].as_signed()),
                    self.clus1.eq(self.in1[8:16].as_signed()),
                    self.clus2.eq(self.in1[16:24].as_signed()),
                    self.clus3.eq(self.in1[24:31].as_signed()),
                 ]
    with m.If(~self.enable):
      m.d.comb += [ self.clus0.eq(self.clus0),
                    self.clus1.eq(self.clus1),
                    self.clus2.eq(self.clus2),
                    self.clus3.eq(self.clus3),
                  ]

    with m.If(self.clear):
      m.d.comb += [ self.clus0.eq(0),
                    self.clus1.eq(0),
                    self.clus2.eq(0),
                    self.clus3.eq(0),
                  ]    

class SIMD8_StoreCodebookTest(TestBase):
  def create_dut(self):
    return SIMD8_StoreCodebook()

  def pack(*values, bits=8):
      mask = (1 << bits) - 1
      result = 0
      for i, v in enumerate(values):
        if v < 0:
          v = mask + v + 1
        result += (v & mask) << (i * bits)
      return result
      
  def test(self):
    
    
    TEST_CASE = [
        # (enable, clear, in1, (clus0, clus1, clus2, clus3))
        (1, 0, pack(64, -6, 10 ,-22),(64, -6, 10 ,-22)), # store the code book (0x03FA0AEA)
        (0, 0, pack(54, -44, 3, -12),(64, -6, 10 ,-22)), # output shouldnt change since enable == 0

        (1, 0, pack(127, -127, 0, -17), (127, -127, 0, -17)), # store new code book corner cases
        (0, 1, pack(54, -44, 3, -12),(0, 0, 0 , 0)), #clear the store book
    ]
    def process():
      for (en, rst, b, exp_clusters) in TEST_CASE:
        yield self.dut.enable.eq(en)
        yield self.dut.clear.eq(rst)
        yield self.dut.in1.eq(b)
        yield Delay(0.1)

        self.assertEqual(exp_clusters, ((yield self.dut.clus0), (yield self.dut.clus1), (yield self.dut.clus2), (yield self.dut.clus3)))
        yield
    self.run_sim(process)

runTests(SIMD8_StoreCodebookTest)

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


In [ ]:
class SIMD8_PushWeight(SimpleElaboratable):
  def __init__(self):
    self.in0 = Signal(32)
    self.in1 = Signal(32)
    self.enable = Signal()


In [156]:
def pack(*values, bits=8):
  mask = (1 << bits) - 1
  result = 0
  for i, v in enumerate(values):
    if v < 0:
      v = mask + v + 1
    result += (v & mask) << (i * bits)
  return result

hex(pack(-100))


'0x9c'